In [3]:
file = open("example.txt","w")
file.write("hello")
y = 0
k = 5 / y
file.close()

ZeroDivisionError: division by zero

In [5]:
file.write("Hi!")

3

In [6]:
file.close()

In [9]:
try:
    file = open("example.txt","w")
    file.write("hello")
    y = 0
    k = 5 / y
    file.close()
except Exception as e:
    print(e)
    if file:
        file.close()

division by zero


In [10]:
file

<_io.TextIOWrapper name='example.txt' mode='w' encoding='UTF-8'>

In [11]:
try:
    file = open("example.txt","w")
    file.write("hello")
    y = 0
    k = 5 / y
except Exception as e:
    print(e)
finally:
    if file:
        file.close()

division by zero


In [14]:
try:
    with open("example.txt","w") as file:
        file.write("hello")
        y = 0
        k = 5 / y
except Exception as e:
    print(e)

division by zero


In [16]:
class SimpleContextManager:
    """A simple context manager to understand the protocol."""
    
    def __init__(self, name):
        self.name = name
        print(f"Created context manager: {self.name}")
    
    def __enter__(self):
        print(f"Entering context: {self.name}")
        print("Setting up resources...")
        return f"resource_for_{self.name}"  # This is what 'as' captures
    
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Exiting context: {self.name}")
        
        if exc_type is not None:
            print(f"An exception occurred: {exc_type.__name__}: {exc_value}")
            print("Cleaning up after exception...")
        else:
            print("Normal cleanup...")
        
        print("Resources cleaned up!")
        # Return False to let exceptions propagate
        return False

print("=== Context Manager Flow ===")
try:
    with SimpleContextManager("demo") as resource:
        print(f"Inside context, using: {resource}")
        print("Doing some work...")
        # Uncomment to see exception handling
        raise ValueError("Oops!")
        print("Work completed!")
    print("After context manager")
    
except Exception as e:
    print(f"Caught exception: {e}")

=== Context Manager Flow ===
Created context manager: demo
Entering context: demo
Setting up resources...
Inside context, using: resource_for_demo
Doing some work...
Exiting context: demo
An exception occurred: ValueError: Oops!
Cleaning up after exception...
Resources cleaned up!
Caught exception: Oops!


In [18]:
from contextlib import contextmanager

# Traditional class-based approach (verbose)

class TimerClass:
    def __init__(self, name):
        self.name = name
        self.start_time = None
    
    def __enter__(self):
        import time
        print(f"Starting timer: {self.name}")
        self.start_time = time.time()
        return self.start_time
    
    def __exit__(self, exc_type, exc_value, traceback):
        import time
        end_time = time.time()
        duration = end_time - self.start_time
        print(f"Timer {self.name}: {duration:.4f} seconds")
        return False

with TimerClass("class_timer") as start:
    import time
    time.sleep(1)   


Starting timer: class_timer
Timer class_timer: 1.0001 seconds


In [20]:
@contextmanager
def timer(name):
    import time
    print(f"Starting timer: {name}")
    start_time = time.time()
    try:
        yield start_time #as x 
    finally:
        end_time = time.time()          
        duration = end_time - start_time
        print(f"Timer {name}: {duration:.4f} seconds")  

with timer("class_timer") as start:
    import time
    time.sleep(1)   


Starting timer: class_timer
Timer class_timer: 1.0001 seconds


In [23]:
@contextmanager
def timer(name):
    import time
    print(f"Starting timer: {name}")
    start_time = time.time()
    yield 
    end_time = time.time()          
    duration = end_time - start_time
    print(f"Timer {name}: {duration:.4f} seconds")  

with timer("class_timer"):
    import time
    time.sleep(1)   


Starting timer: class_timer
Timer class_timer: 1.0001 seconds


In [26]:
@contextmanager
def config_override(**kwargs):
    """Context manager for temporarily overriding configuration."""
    import os
    
    print("Setting up configuration overrides...")
    
    # Store original values
    original_values = {}
    for key, value in kwargs.items():
        original_values[key] = os.environ.get(key)
        os.environ[key] = str(value)
        print(f"  Set {key} = {value}")
    
    try:
        yield  # No specific value needed
    finally:
        print("Restoring original configuration...")
        
        # Restore original values
        for key, original_value in original_values.items():
            if original_value is None:
                if key in os.environ:
                    del os.environ[key]
                    print(f"  Removed {key}")
            else:
                os.environ[key] = original_value
                print(f"  Restored {key} = {original_value}")


In [28]:
import os

print("=== Before Configuration Override Example ===")
print(f"DEBUG before: {os.environ.get('DEBUG', 'Not set')}")
print(f"LOG_LEVEL before: {os.environ.get('LOG_LEVEL', 'Not set')}")

with config_override(DEBUG="True", LOG_LEVEL="INFO"):
    print(f"DEBUG during: {os.environ.get('DEBUG')}")
    print(f"LOG_LEVEL during: {os.environ.get('LOG_LEVEL')}")

print("=== After Configuration Override Example ===")
print(f"DEBUG before: {os.environ.get('DEBUG', 'Not set')}")
print(f"LOG_LEVEL before: {os.environ.get('LOG_LEVEL', 'Not set')}")


=== Before Configuration Override Example ===
DEBUG before: Not set
LOG_LEVEL before: Not set
Setting up configuration overrides...
  Set DEBUG = True
  Set LOG_LEVEL = INFO
DEBUG during: True
LOG_LEVEL during: INFO
Restoring original configuration...
  Removed DEBUG
  Removed LOG_LEVEL
=== After Configuration Override Example ===
DEBUG before: Not set
LOG_LEVEL before: Not set


In [30]:
@contextmanager
def ignore_errors(*exception_types):
    """Context manager that ignores specified exception types."""
    try:
        yield
    except exception_types as e:
        print(f"Ignoring {type(e).__name__}: {e}")
        # By not re-raising, we suppress the exception


In [33]:
@contextmanager
def log_and_reraise_errors(operation_name="operation"):
    """Context manager that logs errors but doesn't suppress them."""
    try:
        yield
    except Exception as e:
        print(f"Error in {operation_name}: {type(e).__name__}: {e}")
        print(f"Re-raising exception for proper handling...")
        raise  # Re-raise the exception


In [34]:
print("1. Ignoring specific errors:")
with ignore_errors(ValueError, TypeError):
    print("Attempting to convert 'abc' to int...")
    result = int("abc")  # This would normally raise ValueError
    print("This won't be reached")

print("Execution continues after ignored error!")

print("\n2. Logging errors without suppressing:")
try:
    with log_and_reraise_errors("data processing"):
        print("Processing data...")
        raise RuntimeError("Processing failed!")
        
except RuntimeError as e:
    print(f"Caught and handled: {e}")


1. Ignoring specific errors:
Attempting to convert 'abc' to int...
Ignoring ValueError: invalid literal for int() with base 10: 'abc'
Execution continues after ignored error!

2. Logging errors without suppressing:
Processing data...
Error in data processing: RuntimeError: Processing failed!
Re-raising exception for proper handling...
Caught and handled: Processing failed!


In [36]:
@contextmanager
def change_directory(path):
    """Context manager to temporarily change working directory."""
    import os
    
    original_dir = os.getcwd()
    print(f"Changing directory from {original_dir} to {path}")
    
    try:
        os.chdir(path)
        yield path
    finally:
        os.chdir(original_dir)
        print(f"Restored directory to {original_dir}")


In [39]:
with change_directory(".."):
    with open("tmp.txt","w") as f:
        f.write("hello")



Changing directory from /workspaces/advanced-python/topic-06-context-managers to ..
Restored directory to /workspaces/advanced-python/topic-06-context-managers


In [ ]:
@contextmanager
def suppress_output():
    """Context manager to suppress stdout temporarily."""
    import sys
    from io import StringIO
    
    original_stdout = sys.stdout
    print("Suppressing output...")
    
    try:
        sys.stdout = StringIO()  # Redirect to nowhere
        yield
    finally:
        sys.stdout = original_stdout
        print("Output restored!")

In [ ]:
import sqlite3
from contextlib import contextmanager

class DatabaseConnection:
    """Context manager for database connections."""
    
    def __init__(self, db_path):
        self.db_path = db_path
        self.connection = None
        print(f"DatabaseConnection created for: {db_path}")
    
    def __enter__(self):
        print(f"Connecting to database: {self.db_path}")
        try:
            self.connection = sqlite3.connect(self.db_path)
            print(" Database connection established")
            return self.connection
        except Exception as e:
            print(f" Failed to connect to database: {e}")
            raise
    
    def __exit__(self, exc_type, exc_value, traceback):
        if self.connection:
            if exc_type is None:
                print("Closing database connection normally")
            else:
                print(f"Closing database connection after exception: {exc_type.__name__}")
            
            try:
                self.connection.close()
                print(" Database connection closed")
            except Exception as e:
                print(f" Error closing database: {e}")
        
        # Don't suppress exceptions
        return False


In [ ]:
@contextmanager
def database_transaction(db_path):
    """Context manager for database transactions."""
    print(f"Starting database transaction: {db_path}")
    
    conn = sqlite3.connect(db_path)
    
    try:
        # Start transaction
        conn.execute("BEGIN TRANSACTION")
        print(" Transaction started")
        
        yield conn
        
        # If we get here, commit the transaction
        conn.execute("COMMIT")
        print(" Transaction committed")
        
    except Exception as e:
        # If anything goes wrong, rollback
        print(f" Error in transaction: {e}")
        try:
            conn.execute("ROLLBACK")
            print(" Transaction rolled back")
        except Exception as rollback_error:
            print(f" Rollback failed: {rollback_error}")
        raise
        
    finally:
        conn.close()
        print(" Database connection closed")


In [ ]:
with database_transaction(":memory:") as conn:
    cursor = conn.cursor()
    
    # Setup
    cursor.execute("CREATE TABLE accounts (id INTEGER, balance REAL)")
    cursor.execute("INSERT INTO accounts VALUES (1, 100.0)")
    cursor.execute("INSERT INTO accounts VALUES (2, 50.0)")
    
    # Transfer money
    cursor.execute("UPDATE accounts SET balance = balance - 25 WHERE id = 1")
    cursor.execute("UPDATE accounts SET balance = balance + 25 WHERE id = 2")
    
    print("Money transfer completed")
